Spike-timing-dependent synaptic modification induced by natural spike trains  
Trying to apply this STDP rule   

Using all weak E cells, interconnected, and see what happens?

1. use the somewhat naive STDP rule (SONG-ABOTT)

In [1]:
from brian2 import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

SONG-ABBOTT STDP RULE

In [35]:
start_scope()

Ne = 10

Cm = 1*ufarad/cm**2
El = -67*mV
ENa = 50*mV
EK = -100*mV
gl = 0.1*msiemens/cm**2
gNa = 100*msiemens/cm**2
gK = 80*msiemens/cm**2
V_I = -80*mV
V_E = 0*mV

# STDP vars
gmax = 1.0*msiemens/cm**2
gampa0 = 0.01*msiemens/cm**2

# gampa = 0.01*msiemens/cm**2
# ggaba = 0.6*msiemens/cm**2

I = 0*amp*meter**-2

eqs = '''
dv/dt = ( (gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) + Isyn + I))/Cm : volt
Isyn = Iampa : amp*meter**-2
Iampa = gampa * (V_E - v) * sampa_tot : amp*meter**-2
sampa_tot : 1
gampa : siemens/meter**2
#Isyn = Igaba : amp*meter**-2
#Igaba = ggaba/Ni * (V_I - v) * sgaba_tot : amp*meter**-2
#sgaba_tot : 1
m = alpham / (alpham + betam) :1
dn/dt = alphan * (1-n) - betan * n : 1
dh/dt = alphah * (1-h) - betah * h : 1
alpham = (0.32/mV) * (v+54*mV) / (-exp((-v-54*mV) / (4*mV)) + 1)/ms : Hz
betam = (0.28/mV) * (v+27*mV)/ (exp((v+27*mV)/(5*mV)) - 1)/ms : Hz
alphah = (0.128) * exp(-(v+50*mV)/(18*mV))/ms : Hz
betah = 4/(exp((-v-27*mV) / (5*mV)) + 1)/ms : Hz
alphan = (0.032/mV) * (v+52*mV) / (-exp(-(v+52*mV) / (5*mV)) + 1)/ms : Hz
betan = 0.5*exp(-(v+57*mV)/(40*mV))/ms : Hz
'''

Ecells = NeuronGroup(Ne, eqs, threshold='v>0*mV',refractory='v > 0*mV',
                method='euler')
Ecells.v = -60*mV
# STDP gampa start from small value
Ecells.gampa = gampa0

Me = StateMonitor(Ecells, 'v', record=True)

# Synapses
tausampar = 0.2*ms
tausampad = 2*ms
kplus = gampa0/2
kminus = gampa0/3
gdelta = 1e-6 *msiemens/cm**2
tau_plus = 10 *ms
tau_minus = 10*ms

eqs_ampa  = ''' 
sampa_tot_post = sampa : 1 (summed)
dsampa/dt= (1+tanh(v_pre/mV/10))/2 * (1-sampa)/tausampar - sampa/tausampad : 1 (clock-driven)
#w : 1 # synaptic weight
#dApre/dt = -Apre / taupre : 1 (event-driven)
#dApost/dt = -Apost / taupost : 1 (event-driven)
'''

# eqs_pre = '''
# gampa_temp : siemens/meter**2
# gampa_temp -= kminus*exp(t-lastspike_post)
# gampa = clip(gampa_temp, gdelta, gmax)
# '''

# eqs_post = '''
# gampa_temp : siemens/meter**2
# gampa_temp += kplus*exp(lastspike_pre-t)
# gampa = clip(gampa_temp, gdelta, gmax)
# '''

# eqs_pre = '''
# gampa = clip(gampa-kminus*exp((t-lastspike_post)/tau_minus), gdelta, gmax)
# '''

# eqs_post = '''
# gampa = clip(gampa+kplus*exp((lastspike_pre-t)/tau_plus), gdelta, gmax)
# '''

# try boolean
eqs_pre = '''
check1 = int(((t-lastspike_pre)/ms)<=1e-6) :1
gampa = clip(gampa-(check1*kminus*exp((t-lastspike_post)/tau_minus), gdelta, gmax)
'''

eqs_post = '''
check1 = int(((t-lastspike_post)/ms)<=1e-6) :1
gampa = clip(gampa+(check2)*kplus*exp((lastspike_pre-t)/tau_plus), gdelta, gmax)
'''

# # E to E, ampa
C_E_E = Synapses(Ecells, Ecells, model=eqs_ampa, 
                 on_pre = eqs_pre,
                 on_post = eqs_post,
                 method='euler')
# # C_E_I = Synapses(Ecells, Icells, model=eqs_ampa, method='euler')
C_E_E.connect()
# C_E_I.w[:] = 1


mon = StateMonitor(C_E_E, 'gampa',record = True)


# sp_E = SpikeMonitor(Ecells,variables='v')
# sp_I = SpikeMonitor(Icells,variables='v')

In [36]:
run(1*ms,report='stdout')


BrianObjectException: Original error and traceback:
Traceback (most recent call last):
  File "/Users/Jingjin/anaconda3/envs/my-brian2-env/lib/python3.6/site-packages/brian2/core/network.py", line 841, in before_run
    obj.before_run(run_namespace)
  File "/Users/Jingjin/anaconda3/envs/my-brian2-env/lib/python3.6/site-packages/brian2/core/base.py", line 278, in device_override_decorated_function
    return func(*args, **kwds)
  File "/Users/Jingjin/anaconda3/envs/my-brian2-env/lib/python3.6/site-packages/brian2/synapses/synapses.py", line 298, in before_run
    CodeRunner.before_run(self, run_namespace)
  File "/Users/Jingjin/anaconda3/envs/my-brian2-env/lib/python3.6/site-packages/brian2/groups/group.py", line 1115, in before_run
    codeobj_class=self.codeobj_class
  File "/Users/Jingjin/anaconda3/envs/my-brian2-env/lib/python3.6/site-packages/brian2/codegen/codeobject.py", line 225, in create_runner_codeobj
    _, uk, u = analyse_identifiers(v, all_variables, recursive=True)
  File

In [27]:
mon.gampa[1:10]

array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.

In [32]:
test1 = True
int(test1)

1